In [16]:
import pandas as pd
import os
import re
import numpy as np
import pprint

In [17]:
df = pd.read_excel("../data/fomc_dissents_data.xlsx",skiprows=3)

In [50]:
df["Date"] = df["FOMC Meeting"].apply(lambda x:str(x).split(" ")[0])
df['FOMC Votes'] = df['FOMC Votes'].apply(lambda x:0 if np.isnan(x) else x)
print(df.columns)

Index(['Year', 'FOMC Meeting', 'Chair', 'Dissent (Y or N)', 'FOMC Votes',
       'Votes for Action', 'Votes Against Action',
       'Number Governors Dissenting', 'Number Presidents Dissenting',
       'No. Governors for Tighter', 'No. Governors for Easier',
       'No. Presidents for Tighter', 'No. Presidents for Easier',
       'Dissenters Tighter', 'Dissenters Easier',
       'Dissenters Other/Indeterminate', 'Date'],
      dtype='object')


In [51]:

voter_df = pd.DataFrame()

for index,row in df.iterrows():
    voters = []
    num_voters = int(row['FOMC Votes'])
    date_path = '../../../collection/python/data/minutes_raw_text/{}.txt'.format(row['Date'])
    if not os.path.exists(date_path):
        continue
    with open(date_path) as f:
        broken = False
        broken_starts = 0
        broken_ends = 0
        lines = f.readlines()
        
        '''First Check For Broken Title'''

        print("CHECKING USING FRAGMENT HEURISTIC")
        for line in lines[:200]:
            if line.strip():
                if broken_ends==0:
                    title_frag = re.match(r'^(?:PRESENT: |PRESENT. )?(?:Mr.|Ms.|Mt.|Mrs. )$',line.strip())
                    if title_frag:
                        if not broken:
                            broken = True
                        print("Broken Begining")
                        print(title_frag.group(0))
                        title_frag_string = str(title_frag.group(0)).replace("PRESENT: ","")
                        voters.append(title_frag_string)
                        broken_starts+=1
                        continue
                if broken and broken_ends<len(voters):
                    name_fragment = re.match('^[A-Z][a-z][A-Za-z]*',line.strip())
                    if name_fragment:
                        print(name_fragment)
                        if broken_ends==0:
                            print("Found {} Fragments".format(len(voters)))
                            print(voters)
                        print("Broken Ending")
                        print(name_fragment.group(0))
                        voters[broken_ends] = voters[broken_ends]+" "+str(name_fragment.group(0))
                        broken_ends+=1
        
        '''Check using Mr. Regex'''
        
        if len(voters)==0:
            print("CHECKING TITLE REGEX")
            for line in lines[:200]:
                '''Then check for valid input'''
                voter_line = re.findall(r'(?:Mr.|Ms.|Mrs.) [A-Z][a-zA-Z]*',line.strip())
                if voter_line:
                    #print(voter_line)
                    voters.append(voter_line[0])
                    if len(voters)>=num_voters:
                        break
        
        '''Check Last Name Regex'''
        
        if len(voters) == 0:
            print("Checking POST-PRESENT-NAME HEURISTIC")
            found_present = False
            for line in lines[:200]:
                if "PRESENT:" in line.strip() or "PRESENT." in line.strip():
                    found_present = True
                    present_line = line.split(",")[0].strip().replace("PRESENT","")
                    name_text = re.match('[A-Z][a-z]*\s?(?:[A-Z][a-z]*)?',present_line)
                    if name_text:
                        voters.append(name_text.group(0))
                    continue
                if found_present:
                    print(line)
                    name_text = re.match('[A-Z][a-z]*\s?(?:[A-Z][a-z]*)?',line.split(",")[0].strip())
                    if name_text:
                        voters.append(name_text.group(0))
                    if len(voters)>=num_voters:
                        break
    print('Date:{}'.format(row['Date']))
    print("Broken Status:{}".format(broken))
    print("Voter Number:{}".format(num_voters))
    print("Voters Found:{}".format(len(voters)))
    pprint.pprint(voters)
    voter_df = voter_df.append({
        "Date":row['FOMC Meeting'],
        "voters_expected":num_voters,
        "voters_observed":len(voters),
        "Voters":voters if num_voters==len(voters) else None,
    },ignore_index=True)
    print("="*50)
print(voter_df)

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1965-01-12
Broken Status:False
Voter Number:11
Voters Found:11
['Mr. Martin',
 'Mr. Hayes',
 'Mr. Balderston',
 'Mr. Daane',
 'Mr. Hickman',
 'Mr. Mitchell',
 'Mr. Robertson',
 'Mr. Shepardson',
 'Mr. Shuford',
 'Mr. Swan',
 'Mr. Wayne']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1965-02-02
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Martin',
 'Mr. Hayes',
 'Mr. Balderston',
 'Mr. Daane',
 'Mr. Hickman',
 'Mr. Mills',
 'Mr. Mitchell',
 'Mr. Robertson',
 'Mr. Shepardson',
 'Mr. Shuford',
 'Mr. Swan',
 'Mr. Wayne']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 6), match='Martin'>
Found 11 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.'

CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 6), match='Martin'>
Found 12 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Martin
<_sre.SRE_Match object; span=(0, 5), match='Hayes'>
Broken Ending
Hayes
<_sre.SRE_Match object; span=(0, 7), match='Brimmer'>
Broken Ending
Brimmer
<_sre.SRE_Match object; span=(0, 4), match='Clay'>
Broken Ending
Clay
<_sre.SRE_Match object; span=(0, 5), match='Daane'>
Broken Ending
Daane
<_sre.SRE_Match object; span=(0, 7), match='Hickman'>
Broken Ending
Hickman
<_sre.SRE_Match object; span=(0, 5), match='Irons'>
Broken Ending
Irons
<_sre.SRE_Match object; span=(0, 6), match='Maisel'>
Broken Ending
Maisel
<_sre.SRE_Match object; spa

Broken Ending
Mitchell
<_sre.SRE_Match object; span=(0, 9), match='Robertson'>
Broken Ending
Robertson
<_sre.SRE_Match object; span=(0, 8), match='Sherrill'>
Broken Ending
Sherrill
Date:1968-05-28
Broken Status:True
Voter Number:12
Voters Found:12
['Mr. Martin',
 'Mr. Hayes',
 'Mr. Brimmer',
 'Mr. Daane',
 'Mr. Ellis',
 'Mr. Galusha',
 'Mr. Hickman',
 'Mr. Kimbrel',
 'Mr. Maisel',
 'Mr. Mitchell',
 'Mr. Robertson',
 'Mr. Sherrill']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 6), match='Martin'>
Found 12 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Martin
<_sre.SRE_Match object; span=(0, 7), match='Brimmer'>
Broken Ending
Brimmer
<_sre.SRE_Match object; spa

Broken Ending
Robertson
<_sre.SRE_Match object; span=(0, 7), match='Scanlon'>
Broken Ending
Scanlon
<_sre.SRE_Match object; span=(0, 8), match='Sherrill'>
Broken Ending
Sherrill
<_sre.SRE_Match object; span=(0, 7), match='Treiber'>
Broken Ending
Treiber
Date:1969-04-01
Broken Status:True
Voter Number:12
Voters Found:12
['Mr. Martin',
 'Mr. Bopp',
 'Mr. Brimmer',
 'Mr. Clay',
 'Mr. Coldwell',
 'Mr. Daane',
 'Mr. Maisel',
 'Mr. Mitchell',
 'Mr. Robertson',
 'Mr. Scanlon',
 'Mr. Sherrill',
 'Mr. Treiber']
CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 6), match='Martin'>
Found 11 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Martin
<_sre.SRE_Match object; span=(0, 5), match='Hayes'>
Broken

CHECKING USING FRAGMENT HEURISTIC
Broken Begining
PRESENT: Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
Broken Begining
Mr.
<_sre.SRE_Match object; span=(0, 5), match='Burns'>
Found 12 Fragments
['Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.', 'Mr.']
Broken Ending
Burns
<_sre.SRE_Match object; span=(0, 5), match='Hayes'>
Broken Ending
Hayes
<_sre.SRE_Match object; span=(0, 4), match='Bopp'>
Broken Ending
Bopp
<_sre.SRE_Match object; span=(0, 7), match='Brimmer'>
Broken Ending
Brimmer
<_sre.SRE_Match object; span=(0, 4), match='Clay'>
Broken Ending
Clay
<_sre.SRE_Match object; span=(0, 8), match='Coldwell'>
Broken Ending
Coldwell
<_sre.SRE_Match object; span=(0, 5), match='Daane'>
Broken Ending
Daane
<_sre.SRE_Match object; span=(0, 6), match='Maisel'>
Broken Ending
Maisel
<_sre.SRE_Match object; span=

...

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1996-05-21
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Boehne',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. Lindsey',
 'Mr. McTeer',
 'Ms. Phillips',
 'Mr. Stern',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1996-07-03
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Boehne',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. Lindsey',
 'Mr. McTeer',
 'Mr. Meyer',
 'Ms. Phillips',
 'Ms. Rivlin',
 'Mr. Stern',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1996-08-20
Broken Status:False
Voter Number:12
Voters Found:12
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Boehne',
 'Mr. Jordan',
 'Mr. Kelley',
 'Mr. Lindsey',
 'Mr. McTeer',
 'Mr. Meyer',
 'Ms. Phillips',
 'Ms. Rivlin',
 'Mr. Stern',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:1996-09-24
Broken Status:False
Voter Nu

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-06-27
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kelley',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-08-21
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kelley',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-10-02
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonough',
 'Mr. Ferguson',
 'Mr. Gramlich',
 'Mr. Hoenig',
 'Mr. Kelley',
 'Mr. Meyer',
 'Ms. Minehan',
 'Mr. Moskow',
 'Mr. Poole']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2001-11-06
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Greenspan',
 'Mr. McDonou

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2006-03-28
Broken Status:False
Voter Number:11
Voters Found:11
['Mr. Bernanke',
 'Mr. Geithner',
 'Ms. Bies',
 'Mr. Guynn',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Mr. Lacker',
 'Mr. Olson',
 'Ms. Pianalto',
 'Mr. Warsh',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2006-05-10
Broken Status:False
Voter Number:11
Voters Found:11
['Mr. Bernanke',
 'Mr. Geithner',
 'Ms. Bies',
 'Mr. Guynn',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Mr. Lacker',
 'Mr. Olson',
 'Ms. Pianalto',
 'Mr. Warsh',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2006-06-29
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. Geithner',
 'Ms. Bies',
 'Mr. Guynn',
 'Mr. Kohn',
 'Mr. Kroszner',
 'Mr. Lacker',
 'Ms. Pianalto',
 'Mr. Warsh',
 'Ms. Yellen']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2006-08-08
Broken Status:False
Voter Number:10
Voters Found:10
['Mr. Bernanke',
 'Mr. G

CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2010-08-10
Broken Status:False
Voter Number:10
Voters Found:1
['Mr. Hoenig']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Checking POST-PRESENT-NAME HEURISTIC


Ben Bernanke, Chairman 

William C. Dudley, Vice Chairman 

James Bullard 

Elizabeth Duke 

Thomas M. Hoenig 

Sandra Pianalto 

Eric Rosengren 

Daniel K. Tarullo  

Kevin Warsh 

Date:2010-09-21
Broken Status:False
Voter Number:9
Voters Found:9
['Ben Bernanke',
 'William C',
 'James Bullard',
 'Elizabeth Duke',
 'Thomas M',
 'Sandra Pianalto',
 'Eric Rosengren',
 'Daniel K',
 'Kevin Warsh']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Date:2010-11-03
Broken Status:False
Voter Number:11
Voters Found:1
['Mr. Hoenig']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Checking POST-PRESENT-NAME HEURISTIC


Ben Bernanke, Chairman 

William C. Dudley, Vice Chairman 

James Bullard 

Elizabeth Duke 

Thomas M. Hoenig 

Sandra Pianalto 

Sarah Bloom

Date:2017-09-20
Broken Status:False
Voter Number:9
Voters Found:9
['Janet L',
 'William C',
 'Lael Brainard',
 'Charles L',
 'Stanley Fischer',
 'Patrick Harker',
 'Robert S',
 'Neel Kashkari',
 'Jerome H']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Checking POST-PRESENT-NAME HEURISTIC
Janet L. Yellen, Chair 

William C. Dudley, Vice Chairman 

Lael Brainard 

Charles L. Evans  

Patrick Harker 

Robert S. Kaplan 

Neel Kashkari 

Jerome H. Powell 

Randal K. Quarles 

Date:2017-11-01
Broken Status:False
Voter Number:9
Voters Found:9
['Janet L',
 'William C',
 'Lael Brainard',
 'Charles L',
 'Patrick Harker',
 'Robert S',
 'Neel Kashkari',
 'Jerome H',
 'Randal K']
CHECKING USING FRAGMENT HEURISTIC
CHECKING TITLE REGEX
Checking POST-PRESENT-NAME HEURISTIC
Janet L. Yellen, Chair 

William C. Dudley, Vice Chairman 

Lael Brainard 

Charles L. Evans  

Patrick Harker 

Robert S. Kaplan 

Neel Kashkari 

Jerome H. Powell 

Randal K. Quarles 

Date:2017-12-13
Broken Status:False

In [52]:
print(len(voter_df[voter_df["Voters"].isna()]))

voter_errors = voter_df[voter_df["Voters"].isna()].reset_index(drop=True)
print(voter_errors)

22
         Date Voters  voters_expected  voters_observed
0  1966-06-07   None             11.0             12.0
1  1967-11-27   None              9.0             10.0
2  1968-04-19   None             11.0             10.0
3  1968-08-19   None             10.0              9.0
4  1969-11-25   None             11.0             10.0
5  1970-06-23   None             12.0             11.0
6  1971-02-09   None             11.0             12.0
7  1975-02-19   None             11.0             12.0
8  1975-08-19   None             12.0             11.0
9  1978-05-05   None             10.0              4.0
10 1979-05-22   None             10.0              5.0
11 1979-10-06   None             12.0              5.0
12 2010-03-16   None             10.0              1.0
13 2010-04-28   None             10.0              1.0
14 2010-06-23   None             10.0              1.0
15 2010-08-10   None             10.0              1.0
16 2010-11-03   None             11.0              1.0
17 2011

In [53]:
s = voter_df.sample()
print(s)
print(s['Voters'].item())

          Date                                             Voters  \
225 1988-05-17  [Mr. Greenspan, Mr. Corrigan, Mr. Angell, Mr. ...   

     voters_expected  voters_observed  
225             11.0             11.0  
['Mr. Greenspan', 'Mr. Corrigan', 'Mr. Angell', 'Mr. Black', 'Mr. Forrestal', 'Mr. Heller', 'Mr. Hoskins', 'Mr. Johnson', 'Mr. Kelley', 'Mr. Parry', 'Ms. Seger']


In [54]:
voter_errors.to_csv("voting_errors.csv")

SyntaxError: unexpected EOF while parsing (<ipython-input-55-5917a5cb4283>, line 1)